In [ ]:
import kagglehub
import pandas as pd
import glob
from sentence_transformers import SentenceTransformer
import torch
import gradio as gr



In [2]:
path = kagglehub.dataset_download("kunalbhar/house-md-transcripts")


c:\Users\Yakov\Desktop\mipt\nlp\hw1\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 2.15M/2.15M [00:00<00:00, 2.64MB/s]

Extracting files...


In [6]:
seasons = []
for file in glob.glob(path+'\season*.csv'):
    df = pd.read_csv(file, encoding= 'unicode_escape')
    seasons.append(df)
   
all_seasons = pd.concat(seasons)
all_seasons.head()

In [10]:
house_dialogue = []
for i in range(len(all_seasons)-1):
    current = all_seasons.iloc[i]
    next_line = all_seasons.iloc[i+1]
    
    if current['name'] != 'House' and next_line['name'] == 'House':
        house_dialogue.append({
            'question': current['line'],
            'answer': next_line['line']
        })

house_qa = pd.DataFrame(house_dialogue)
house_qa.head()

,question,answer
0,"29 year old female, first seizure one month a...",See that? They all assume I'm a Patient becau...
1,So put on a white coat like the rest of us.,I don't want them to think I'm a doctor.
2,You see where the administration might have a...,People don't want a sick doctor.
3,Fair enough. I don't like healthy Patients. T...,"The one who can't talk, I liked that part."
4,She's my cousin.,And your cousin doesn't like the diagnosis. I...


In [11]:
len(house_qa)

19324

In [23]:
model = SentenceTransformer('all-MiniLM-L6-v2')
question_embeddings = model.encode(house_qa['question'].tolist(), convert_to_tensor=True)

def get_house_response(user_input, n_responses=1):
    input_embedding = model.encode(user_input, convert_to_tensor=True)
    similarities = torch.nn.functional.cosine_similarity(input_embedding.unsqueeze(0), question_embeddings)
    
    most_similar_indices = torch.argsort(similarities, descending=True)[:n_responses]
    
    return house_qa.iloc[most_similar_indices]['answer'].values


In [24]:
def chat_with_house(message, history):
    response = get_house_response(message)[0]
    return response

demo = gr.ChatInterface(
        fn=chat_with_house,
        title="Чат с доктором Хаусом",
)

demo.launch()


c:\Users\Yakov\Desktop\mipt\nlp\hw1\venv\lib\site-packages\gradio\components\chatbot.py:291: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
